# UNIDADE 3:  Regressão Linear

3.1 Previsões simples (Regressão linear)

3.2 Previsões complexas (Regressão linear múltipla)

3.3   Algoritmos de Classificação

3.3.1 Métodos simbólicos: Árvore de Decisão

3.3.2 Métodos ensemble: Random Forest


<div style="border: 2px solid #00008B; padding: 15px; border-radius: 10px; background-color: #00008B; color: #FFFFFF; font-family: Arial;">
  <h1 style="margin-top: 0;"> Predição de Cargas de Trabalho para Juízes</h1>
  <p>Descrição: Prever o volume de trabalho em diferentes tribunais com base em fatores como tamanho da população, tipos de casos comuns na região, etc.
</p>
</div>


<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
  <ul>
    <li>Tamanho da população da região onde o tribunal está localizado (valores real entre 30,000 e 500,000)</li>
    <li>Número de juízes ativos no tribunal (valores inteiros entre 1 e 20)</li>
    <li>Número de casos recebidos mensalmente (valores inteiros entre 150 e 800)</li>
    <li>Distribuição dos tipos de casos comuns na região (valores: criminal, civil, trabalhista)</li>
    <li>Taxa de crescimento da população na região (valores reais entre 0.0 e 1.0)</li>
    <li>Taxa de criminalidade na região (valores reais entre 0.0 e 1.0)</li>
    <li>Média de tempo gasto em cada tipo de caso em meses (número inteiro)</li>
    <li>Nível de automação e eficiência do tribunal (valores inteiros entre 0 e 10)</li>
    <li>Nível de congestionamento do sistema judicial na região (baixo, médio, alto)</li>
    <li>Número de advogados atuando na região (valores inteiros entre 1 e 100)</li>
    <li>Número de prédios judiciais na região (valores inteiros entre 1 e 10)</li>
    <li>Nível de urbanização da região (baixo, médio, alto)</li>
    <li>Nível de educação da população na região (valores reais entre 0.0 e 1.0)</li>
    <li>Número de habitantes por juiz (valor fruto da divisão de tamanho da população da região pelo número de juízes ativos no tribunal)</li>
    <li>Índice de Desenvolvimento Humano (valores reais entre 0.0 e 1.0)</li>
    <li>Média de idade dos  es (entre 35 e 60)</li>
    <li><b>Número de casos pendentes atualmente no tribunal (valores inteiros, coluna que não pode ser nula)</b></li>
  </ul>
</div>




In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.float_format', '{:.5f}'.format)

import streamlit as st
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Ingestão de Dados

In [3]:
df_servidor = pd.read_csv("../data/raw/predicao_cargas_trabalho.csv")

In [4]:
df_servidor.head(5)

,codigo_tribunal,populacao,num_juizes,casos_recebidos,tipo_caso,taxa_crescimento_populacao,taxa_criminalidade,tempo_casos,nivel_automacao,congestionamento,advogados,urbanizacao,nivel_educacao,habitantes_por_juiz,idh,idade_juizes,casos_pendentes
0,2978G,NaN,7.00000,338.40523,trabalhista,0.87225,1.00000,5.00000,4.00000,NaN,18.00000,baixo,0.86473,27199.85714,1.00000,45.00000,1425.89694
1,6959G,175281.00000,20.00000,488.01572,trabalhista,0.54684,0.69849,15.00000,NaN,medio,98.00000,alto,0.80249,8764.05000,0.82631,57.00000,1655.64472
2,3766A,209515.00000,15.00000,322.87380,criminal,1.00000,1.00000,14.00000,3.00000,baixo,12.00000,baixo,0.60650,13967.66667,0.93835,35.00000,1475.27192
3,3637B,45029.00000,11.00000,424.08932,criminal,1.00000,0.74196,9.00000,8.00000,medio,91.00000,medio,1.00000,4093.54545,0.74196,48.00000,1189.20912
4,8725F,490829.00000,8.00000,484.75580,criminal,0.79694,0.65267,20.00000,4.00000,medio,24.00000,alto,1.00000,61353.62500,0.75420,NaN,1389.56877


# Limpeza e transformação de dados

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: codigo_tribunal</h3>
</div>

In [5]:
df_servidor["codigo_tribunal"].unique()

array(['2978G', '6959G', '3766A', ..., '9262H', '1924B', '3032G'],
      dtype=object)

In [6]:
df_servidor = df_servidor.dropna(subset=['codigo_tribunal'])
df_servidor["codigo_tribunal"].value_counts(dropna=False)

codigo_tribunal
4650A    8
5561E    8
3784H    7
8857F    7
3308C    7
        ..
7031G    1
1686G    1
2258D    1
5456G    1
3032G    1
Name: count, Length: 2437, dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: populacao</h3>
</div>

In [7]:
df_servidor["populacao"].unique()
df_servidor['populacao'].fillna(df_servidor['populacao'].mean(), inplace=True)
df_servidor["populacao"].value_counts(dropna=False)

populacao
234400.04540    98
54502.00000      2
147399.00000     2
65897.00000      2
87196.00000      2
                ..
464352.00000     1
54918.00000      1
61625.00000      1
177764.00000     1
264748.00000     1
Name: count, Length: 4778, dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: num_juizes</h3>
</div>

In [8]:
df_servidor["num_juizes"].unique()
df_servidor = df_servidor.dropna(subset=['num_juizes'])
df_servidor["num_juizes"].value_counts(dropna=False)

num_juizes
17.00000    281
15.00000    273
13.00000    263
8.00000     263
3.00000     260
4.00000     248
5.00000     245
19.00000    241
12.00000    239
6.00000     238
20.00000    237
2.00000     237
1.00000     235
11.00000    233
9.00000     229
10.00000    226
18.00000    223
14.00000    216
16.00000    211
7.00000     203
Name: count, dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: casos_recebidos</h3>
</div>

In [9]:
df_servidor["casos_recebidos"].value_counts(dropna=False)
df_servidor['casos_recebidos'].fillna(df_servidor['casos_recebidos'].mean(), inplace=True)
df_servidor["casos_recebidos"].value_counts(dropna=False)

casos_recebidos
477.50106    99
676.00000     2
667.00000     2
356.00000     2
338.40523     1
             ..
670.28461     1
551.36560     1
737.64763     1
662.48741     1
502.00000     1
Name: count, Length: 4700, dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: tipo_caso</h3>
</div>

In [10]:
df_servidor["tipo_caso"].unique()
df_servidor['tipo_caso'].fillna('civil', inplace=True)
df_servidor["tipo_caso"].value_counts(dropna=False)

tipo_caso
civil          1652
criminal       1576
trabalhista    1573
Name: count, dtype: int64

In [11]:
categorias = df_servidor['tipo_caso'].astype('category').cat.categories
codigos = df_servidor['tipo_caso'].astype('category').cat.codes
print({codigo: categoria for codigo, categoria in enumerate(categorias)})

{0: 'civil', 1: 'criminal', 2: 'trabalhista'}


In [12]:
df_servidor['tipo_caso'] = df_servidor['tipo_caso'].astype('category').cat.codes
df_servidor["tipo_caso"].unique()

array([2, 1, 0], dtype=int8)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: taxa_crescimento_populacao</h3>
</div>

In [13]:
df_servidor["taxa_crescimento_populacao"].value_counts(dropna=False)

taxa_crescimento_populacao
0.00000    362
1.00000    353
NaN         96
0.87225      1
0.29123      1
          ... 
0.03793      1
0.40552      1
0.26367      1
0.28400      1
0.72438      1
Name: count, Length: 3993, dtype: int64

In [14]:
df_servidor = df_servidor.dropna(subset=['taxa_crescimento_populacao'])
df_servidor["taxa_crescimento_populacao"].value_counts(dropna=False)

taxa_crescimento_populacao
0.00000    362
1.00000    353
0.87225      1
0.29123      1
0.44349      1
          ... 
0.03793      1
0.40552      1
0.26367      1
0.28400      1
0.72438      1
Name: count, Length: 3992, dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: tempo_casos</h3>
</div>

In [15]:
df_servidor["tempo_casos"].unique()

array([ 5., 15., 14.,  9., 20.,  4., 21., 22.,  7., 17., 18., 19., 10.,
       13.,  2., 11.,  6.,  3.,  1., 12.,  8., 23., 16., nan])

In [16]:
df_servidor = df_servidor.dropna(subset=['tempo_casos'])
df_servidor["tempo_casos"].value_counts(dropna=False)

tempo_casos
3.00000     234
21.00000    233
9.00000     223
12.00000    222
15.00000    218
5.00000     215
23.00000    212
10.00000    208
13.00000    206
22.00000    199
16.00000    198
18.00000    198
17.00000    198
4.00000     197
11.00000    196
8.00000     194
19.00000    193
6.00000     187
1.00000     187
14.00000    179
20.00000    178
7.00000     172
2.00000     167
Name: count, dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: nivel_automacao</h3>
</div>

In [17]:
df_servidor["nivel_automacao"].unique()

array([ 4., nan,  3.,  8.,  2., 10.,  7.,  0.,  5.,  9.,  1.,  6.])

In [18]:
from sklearn.impute import SimpleImputer

# Imputação com o mais frequente
imputer = SimpleImputer(strategy='most_frequent')
df_servidor['nivel_automacao'] = imputer.fit_transform(df_servidor[['nivel_automacao']])

In [19]:
#df_servidor['nivel_automacao'].fillna(5, inplace=True)

In [20]:
df_servidor["nivel_automacao"].value_counts(dropna=False)

nivel_automacao
6.00000     535
4.00000     431
9.00000     427
10.00000    425
5.00000     424
8.00000     412
2.00000     404
1.00000     395
3.00000     394
7.00000     386
0.00000     381
Name: count, dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: congestionamento</h3>
</div>

In [21]:
df_servidor["congestionamento"].unique()

array([nan, 'medio', 'baixo', 'alto', 'invalido'], dtype=object)

In [22]:
df_servidor['congestionamento'].fillna('invalido', inplace=True)
df_servidor["congestionamento"].value_counts(dropna=False)

congestionamento
medio       1469
alto        1469
baixo       1424
invalido     252
Name: count, dtype: int64

In [23]:
categorias = df_servidor['congestionamento'].astype('category').cat.categories
codigos = df_servidor['congestionamento'].astype('category').cat.codes
print({codigo: categoria for codigo, categoria in enumerate(categorias)})

{0: 'alto', 1: 'baixo', 2: 'invalido', 3: 'medio'}


In [24]:
df_servidor['congestionamento'] = df_servidor['congestionamento'].astype('category').cat.codes
df_servidor["congestionamento"].unique()

array([2, 3, 1, 0], dtype=int8)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: advogados</h3>
</div>

In [25]:
df_servidor["advogados"].unique()

array([18., 98., 12., 91., 24., 97., 79., 43., 32., 81., 99., 47., nan,
       35., 77.,  9., 65., 59., 89., 86., 10., 90., 19., 50.,  4.,  5.,
       29., 60., 28., 83.,  1., 51., 36., 64., 14., 11., 41., 25., 17.,
        8., 58.,  2., 27., 13., 63., 93., 26., 69., 39., 92., 37., 67.,
       68., 75., 34., 70., 33., 72., 84., 82., 48., 71., 22., 16.,  7.,
       44., 55.,  6., 40., 74., 46., 96., 73., 76., 61., 78., 94., 31.,
        3., 54., 62., 87., 49., 45., 42., 21., 66., 88., 23., 15., 80.,
       38., 56., 52., 53., 30., 85., 57., 20., 95.])

In [26]:
df_servidor = df_servidor.dropna(subset=['advogados'])
df_servidor["advogados"].value_counts(dropna=False)

advogados
22.00000    73
8.00000     71
4.00000     70
24.00000    69
12.00000    63
            ..
73.00000    30
75.00000    29
78.00000    29
72.00000    29
54.00000    28
Name: count, Length: 99, dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: urbanizacao</h3>
</div>

In [27]:
df_servidor["urbanizacao"].unique()

array(['baixo', 'alto', 'medio', 'desconhecido', nan], dtype=object)

In [28]:
df_servidor['urbanizacao'].fillna('desconhecido', inplace=True)
df_servidor["urbanizacao"].value_counts(dropna=False)

urbanizacao
alto            1432
baixo           1428
medio           1410
desconhecido     255
Name: count, dtype: int64

In [29]:
categorias = df_servidor['urbanizacao'].astype('category').cat.categories
codigos = df_servidor['urbanizacao'].astype('category').cat.codes
print({codigo: categoria for codigo, categoria in enumerate(categorias)})

{0: 'alto', 1: 'baixo', 2: 'desconhecido', 3: 'medio'}


In [30]:
df_servidor['urbanizacao'] = df_servidor['urbanizacao'].astype('category').cat.codes
df_servidor["urbanizacao"].unique()

array([1, 0, 3, 2], dtype=int8)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: nivel_educacao</h3>
</div>

In [31]:
df_servidor["nivel_educacao"].unique()

array([0.86472873, 0.80249159, 0.60650273, ..., 0.50786283, 0.90757782,
       0.48493341])

In [32]:
df_servidor["nivel_educacao"].value_counts(dropna=False)
df_servidor['nivel_educacao'].fillna(df_servidor['nivel_educacao'].mean(), inplace=True)
df_servidor["nivel_educacao"].value_counts(dropna=False)

nivel_educacao
0.00000    352
1.00000    345
0.50114     92
0.86473      1
0.60239      1
          ... 
0.74062      1
0.83977      1
0.24977      1
0.66865      1
0.48493      1
Name: count, Length: 3739, dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: habitantes_por_juiz</h3>
</div>

In [33]:
df_servidor["habitantes_por_juiz"].unique()

array([27199.85714286,  8764.05      , 13967.66666667, ...,
       14244.29411765, 24131.5       , 15573.41176471])

In [34]:
df_servidor = df_servidor.dropna(subset=['habitantes_por_juiz'])
df_servidor["habitantes_por_juiz"].value_counts(dropna=False)

habitantes_por_juiz
49692.22222    2
36533.00000    2
74073.50000    2
32948.50000    2
29520.00000    2
              ..
12093.33333    1
13051.13333    1
13664.07143    1
54030.85714    1
15573.41176    1
Name: count, Length: 4422, dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: idh</h3>
</div>

In [35]:
df_servidor["idh"].unique()

array([1.        , 0.82631083, 0.93835172, ..., 0.87933949, 0.03390716,
       0.88361107])

In [36]:
df_servidor["idh"].value_counts(dropna=False)
df_servidor['idh'].fillna(df_servidor['idh'].mean(), inplace=True)
df_servidor["idh"].value_counts(dropna=False)

idh
1.00000    135
0.54381     92
0.00000     48
0.05080      1
0.02354      1
          ... 
0.87468      1
0.67193      1
0.76181      1
0.49652      1
0.88361      1
Name: count, Length: 4159, dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: idade_juizes</h3>
</div>

In [37]:
df_servidor["idade_juizes"].unique()

array([45., 57., 35., 48., nan, 56., 39., 55., 59., 43., 44., 80., 47.,
       52., 49., 46., 54., 53., 41., 38., 50., 60., 40., 36., 58., 42.,
       37., 51., 72., 68., 65., 74., 70., 79., 78., 83., 69., 81., 75.,
       66., 77., 73., 84., 82., 71.])

In [38]:
df_servidor["idade_juizes"].value_counts(dropna=False)
df_servidor['idade_juizes'].fillna(df_servidor['idade_juizes'].mean(), inplace=True)
df_servidor["idade_juizes"].value_counts(dropna=False)

idade_juizes
54.00000    199
39.00000    179
60.00000    177
52.00000    177
49.00000    176
51.00000    176
43.00000    174
38.00000    173
58.00000    173
35.00000    169
46.00000    169
56.00000    167
50.00000    164
59.00000    164
40.00000    163
47.00000    162
53.00000    160
36.00000    154
57.00000    154
48.00000    153
44.00000    150
45.00000    149
55.00000    145
37.00000    145
41.00000    144
42.00000    143
48.16694     88
68.00000      9
65.00000      9
80.00000      6
73.00000      6
81.00000      6
78.00000      6
79.00000      5
74.00000      5
72.00000      5
71.00000      5
75.00000      4
66.00000      4
70.00000      3
83.00000      3
69.00000      2
77.00000      2
84.00000      2
82.00000      2
Name: count, dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: casos_pendentes</h3>
</div>

In [39]:
df_servidor["casos_pendentes"].unique()

array([1425.89694235, 1655.6447161 , 1475.27192218, ...,  249.        ,
       1065.        ,  996.        ])

In [40]:
df_servidor = df_servidor.dropna(subset=['casos_pendentes'])
df_servidor["casos_pendentes"].value_counts(dropna=False)

casos_pendentes
1544.00000    2
493.00000     2
688.00000     2
1084.49109    1
961.25119     1
             ..
1397.16216    1
944.10540     1
284.84142     1
2699.93076    1
996.00000     1
Name: count, Length: 4336, dtype: int64

In [41]:
df_servidor.isna().sum()

codigo_tribunal               0
populacao                     0
num_juizes                    0
casos_recebidos               0
tipo_caso                     0
taxa_crescimento_populacao    0
taxa_criminalidade            0
tempo_casos                   0
nivel_automacao               0
congestionamento              0
advogados                     0
urbanizacao                   0
nivel_educacao                0
habitantes_por_juiz           0
idh                           0
idade_juizes                  0
casos_pendentes               0
dtype: int64

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: idade_juizes</h3>
</div>

In [42]:
bins = [18, 30, 40, 50, 60, 75, 80, 85]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89']
df_servidor['idade_juizes'] = pd.cut(df_servidor['idade_juizes'], bins=bins, labels=labels, right=True, include_lowest=True)
df_servidor["idade_juizes"].unique()

['40-49', '50-59', '30-39', '70-79', '60-69', '80-89']
Categories (7, object): ['18-29' < '30-39' < '40-49' < '50-59' < '60-69' < '70-79' < '80-89']

In [43]:
df_servidor["idade_juizes"].value_counts(dropna=False)

idade_juizes
50-59    1653
40-49    1639
30-39     965
60-69      51
70-79      19
80-89      12
18-29       0
Name: count, dtype: int64

In [44]:
df_servidor = pd.get_dummies(df_servidor, columns=['idade_juizes'], prefix='idade_juizes')
dummy_columns = [col for col in df_servidor.columns if col.startswith('idade_juizes_')]
df_servidor[dummy_columns] = df_servidor[dummy_columns].astype(int)
df_servidor

,codigo_tribunal,populacao,num_juizes,casos_recebidos,tipo_caso,taxa_crescimento_populacao,taxa_criminalidade,tempo_casos,nivel_automacao,congestionamento,...,habitantes_por_juiz,idh,casos_pendentes,idade_juizes_18-29,idade_juizes_30-39,idade_juizes_40-49,idade_juizes_50-59,idade_juizes_60-69,idade_juizes_70-79,idade_juizes_80-89
0,2978G,234400.04540,7.00000,338.40523,2,0.87225,1.00000,5.00000,4.00000,2,...,27199.85714,1.00000,1425.89694,0,0,1,0,0,0,0
1,6959G,175281.00000,20.00000,488.01572,2,0.54684,0.69849,15.00000,6.00000,3,...,8764.05000,0.82631,1655.64472,0,0,0,1,0,0,0
2,3766A,209515.00000,15.00000,322.87380,1,1.00000,1.00000,14.00000,3.00000,1,...,13967.66667,0.93835,1475.27192,0,1,0,0,0,0,0
3,3637B,45029.00000,11.00000,424.08932,1,1.00000,0.74196,9.00000,8.00000,3,...,4093.54545,0.74196,1189.20912,0,0,1,0,0,0,0
4,8725F,490829.00000,8.00000,484.75580,1,0.79694,0.65267,20.00000,4.00000,3,...,61353.62500,0.75420,1389.56877,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,5840F,64381.00000,3.00000,728.00000,0,0.66371,0.96175,22.00000,6.00000,0,...,21460.33333,0.21590,1261.00000,0,1,0,0,0,0,0
4996,1924B,403595.00000,10.00000,536.00000,1,0.42938,0.45139,7.00000,5.00000,3,...,40359.50000,0.57341,1379.00000,0,0,1,0,0,0,0
4997,4944H,242153.00000,17.00000,561.00000,2,0.84879,0.56073,23.00000,5.00000,1,...,14244.29412,0.87934,249.00000,0,0,0,1,0,0,0
4998,3555D,48263.00000,2.00000,519.00000,1,0.77827,0.58441,11.00000,10.00000,3,...,24131.50000,0.03391,1065.00000,0,0,0,1,0,0,0


# Ajustes de tipos

In [45]:
df_servidor['casos_pendentes'] = df_servidor['casos_pendentes'].astype(int)
df_servidor['casos_recebidos'] = df_servidor['casos_recebidos'].astype(int)

In [46]:
df_servidor.drop(columns=['codigo_tribunal'], inplace=True)

In [47]:
df_servidor.isna().sum()

populacao                     0
num_juizes                    0
casos_recebidos               0
tipo_caso                     0
taxa_crescimento_populacao    0
taxa_criminalidade            0
tempo_casos                   0
nivel_automacao               0
congestionamento              0
advogados                     0
urbanizacao                   0
nivel_educacao                0
habitantes_por_juiz           0
idh                           0
casos_pendentes               0
idade_juizes_18-29            0
idade_juizes_30-39            0
idade_juizes_40-49            0
idade_juizes_50-59            0
idade_juizes_60-69            0
idade_juizes_70-79            0
idade_juizes_80-89            0
dtype: int64

In [48]:
df_servidor.shape

(4339, 22)

In [49]:
df_servidor.head(5)

,populacao,num_juizes,casos_recebidos,tipo_caso,taxa_crescimento_populacao,taxa_criminalidade,tempo_casos,nivel_automacao,congestionamento,advogados,...,habitantes_por_juiz,idh,casos_pendentes,idade_juizes_18-29,idade_juizes_30-39,idade_juizes_40-49,idade_juizes_50-59,idade_juizes_60-69,idade_juizes_70-79,idade_juizes_80-89
0,234400.04540,7.00000,338,2,0.87225,1.00000,5.00000,4.00000,2,18.00000,...,27199.85714,1.00000,1425,0,0,1,0,0,0,0
1,175281.00000,20.00000,488,2,0.54684,0.69849,15.00000,6.00000,3,98.00000,...,8764.05000,0.82631,1655,0,0,0,1,0,0,0
2,209515.00000,15.00000,322,1,1.00000,1.00000,14.00000,3.00000,1,12.00000,...,13967.66667,0.93835,1475,0,1,0,0,0,0,0
3,45029.00000,11.00000,424,1,1.00000,0.74196,9.00000,8.00000,3,91.00000,...,4093.54545,0.74196,1189,0,0,1,0,0,0,0
4,490829.00000,8.00000,484,1,0.79694,0.65267,20.00000,4.00000,3,24.00000,...,61353.62500,0.75420,1389,0,0,1,0,0,0,0


In [50]:
# Salvar o DataFrame como um arquivo CSV
df_servidor.to_csv('../data/interim/predicao_cargas_trabalho_processado.csv', index=False)

# Normalização e Padronização dos dados

### Padronização (Z-score Scaling)

$z = \frac{x - \mu}{\sigma}$


In [51]:
scaler = StandardScaler()
df_normalizado = pd.DataFrame(scaler.fit_transform(df_servidor), columns=df_servidor.columns)
df_normalizado

,populacao,num_juizes,casos_recebidos,tipo_caso,taxa_crescimento_populacao,taxa_criminalidade,tempo_casos,nivel_automacao,congestionamento,advogados,...,habitantes_por_juiz,idh,casos_pendentes,idade_juizes_18-29,idade_juizes_30-39,idade_juizes_40-49,idade_juizes_50-59,idade_juizes_60-69,idade_juizes_70-79,idade_juizes_80-89
0,0.00106,-0.61744,-0.66145,1.24541,1.19484,1.87040,-1.07338,-0.35178,0.51108,-0.96598,...,-0.07855,1.91810,0.00267,0.00000,-0.53480,1.28349,-0.78448,-0.10906,-0.06632,-0.05266
1,-0.41671,1.62910,0.04887,1.24541,0.15494,0.64084,0.43984,0.29253,1.32607,1.81440,...,-0.91841,1.18770,0.33086,0.00000,-0.53480,-0.77913,1.27473,-0.10906,-0.06632,-0.05266
2,-0.17479,0.76504,-0.73721,0.02172,1.60307,1.87040,0.28852,-0.67393,-0.30391,-1.17451,...,-0.68135,1.65885,0.07401,0.00000,1.86986,-0.77913,-0.78448,-0.10906,-0.06632,-0.05266
3,-1.33713,0.07380,-0.25420,0.02172,1.60307,0.81813,-0.46809,0.93684,1.32607,1.57112,...,-1.13117,0.83298,-0.33408,0.00000,-0.53480,1.28349,-0.78448,-0.10906,-0.06632,-0.05266
4,1.81311,-0.44463,0.02993,0.02172,0.95415,0.45401,1.19645,-0.35178,1.32607,-0.75745,...,1.47735,0.88446,-0.04870,0.00000,-0.53480,1.28349,-0.78448,-0.10906,-0.06632,-0.05266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4334,-1.20038,-1.30869,1.18538,-1.20198,0.52842,1.71442,1.49910,0.29253,-1.11889,-0.79221,...,-0.34002,-1.37920,-0.23134,0.00000,1.86986,-0.77913,-0.78448,-0.10906,-0.06632,-0.05266
4335,1.19667,-0.09901,0.27618,0.02172,-0.22041,-0.36680,-0.77074,-0.02962,1.32607,0.98029,...,0.52095,0.12418,-0.06297,0.00000,-0.53480,1.28349,-0.78448,-0.10906,-0.06632,-0.05266
4336,0.05584,1.11067,0.39456,1.24541,1.11985,0.07908,1.65042,-0.02962,-0.30391,-0.82696,...,-0.66875,1.41069,-1.67536,0.00000,-0.53480,-0.77913,1.27473,-0.10906,-0.06632,-0.05266
4337,-1.31428,-1.48150,0.19567,0.02172,0.89449,0.17567,-0.16545,1.58115,1.32607,0.04191,...,-0.21833,-2.14453,-0.51101,0.00000,-0.53480,-0.77913,1.27473,-0.10906,-0.06632,-0.05266


### Normalização (Min-Max Scaling)

$z = \frac{x - \min(x)}{\max(x) - \min(x)}$

In [52]:
scaler = MinMaxScaler()
df_normalizado = pd.DataFrame(scaler.fit_transform(df_servidor), columns=df_servidor.columns)
df_normalizado

,populacao,num_juizes,casos_recebidos,tipo_caso,taxa_crescimento_populacao,taxa_criminalidade,tempo_casos,nivel_automacao,congestionamento,advogados,...,habitantes_por_juiz,idh,casos_pendentes,idade_juizes_18-29,idade_juizes_30-39,idade_juizes_40-49,idade_juizes_50-59,idade_juizes_60-69,idade_juizes_70-79,idade_juizes_80-89
0,0.43482,0.31579,0.41271,1.00000,0.87225,1.00000,0.18182,0.40000,0.66667,0.17347,...,0.20807,1.00000,0.49684,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000
1,0.30901,1.00000,0.53339,1.00000,0.54684,0.69849,0.63636,0.60000,1.00000,0.98980,...,0.05870,0.82631,0.54882,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000
2,0.38186,0.73684,0.39984,0.50000,1.00000,1.00000,0.59091,0.30000,0.33333,0.11224,...,0.10086,0.93835,0.50814,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.03183,0.52632,0.48190,0.50000,1.00000,0.74196,0.36364,0.80000,1.00000,0.91837,...,0.02087,0.74196,0.44349,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000
4,0.98052,0.36842,0.53017,0.50000,0.79694,0.65267,0.86364,0.40000,1.00000,0.23469,...,0.48477,0.75420,0.48870,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4334,0.07301,0.10526,0.72647,0.00000,0.66371,0.96175,0.95455,0.60000,0.00000,0.22449,...,0.16157,0.21590,0.45976,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4335,0.79488,0.47368,0.57200,0.50000,0.42938,0.45139,0.27273,0.50000,1.00000,0.74490,...,0.31468,0.57341,0.48644,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000
4336,0.45132,0.84211,0.59212,1.00000,0.84879,0.56073,1.00000,0.50000,0.33333,0.21429,...,0.10310,0.87934,0.23101,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000
4337,0.03871,0.05263,0.55833,0.50000,0.77827,0.58441,0.45455,1.00000,1.00000,0.46939,...,0.18321,0.03391,0.41546,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000


In [53]:
# Salvar o scaler ajustado
joblib.dump(scaler, '../models/scaler.pkl')

['../models/scaler.pkl']

In [54]:
# Salvar o DataFrame como um arquivo CSV
df_normalizado.to_csv('../data/processed/predicao_cargas_trabalho_normalizado.csv', index=False)